In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, explode, struct, lit, median, when
from pyspark.sql.types import DoubleType

# Initialize Spark Session (if not already active)
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("HousingDataProcessor").getOrCreate()



In [ ]:
# Upload data
from google.colab import files
uploaded = files.upload()

Saving hpi_at_zip5.xlsx to hpi_at_zip5.xlsx


In [ ]:
uploaded = files.upload()

Saving geonames-postal-code.csv to geonames-postal-code.csv


In [ ]:
# Load the Housing Data
housing_df = spark.read.csv("HousingData.csv", header=True, inferSchema=True)
housing_df.show(5)

+--------+--------+----------+----------+---------+-----+-----------+--------------------+---------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-

In [ ]:
housing_df = housing_df.drop("RegionID", "State", "CountyName", "SizeRank","RegionType","StateName","City","Metro")
housing_df.show(5)

+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+

In [ ]:
housing_df =  housing_df.withColumnRenamed("RegionName", "ZIP")
housing_df.printSchema()

root
 |-- ZIP: integer (nullable = true)
 |-- 2000-01-31: double (nullable = true)
 |-- 2000-02-29: double (nullable = true)
 |-- 2000-03-31: double (nullable = true)
 |-- 2000-04-30: double (nullable = true)
 |-- 2000-05-31: double (nullable = true)
 |-- 2000-06-30: double (nullable = true)
 |-- 2000-07-31: double (nullable = true)
 |-- 2000-08-31: double (nullable = true)
 |-- 2000-09-30: double (nullable = true)
 |-- 2000-10-31: double (nullable = true)
 |-- 2000-11-30: double (nullable = true)
 |-- 2000-12-31: double (nullable = true)
 |-- 2001-01-31: double (nullable = true)
 |-- 2001-02-28: double (nullable = true)
 |-- 2001-03-31: double (nullable = true)
 |-- 2001-04-30: double (nullable = true)
 |-- 2001-05-31: double (nullable = true)
 |-- 2001-06-30: double (nullable = true)
 |-- 2001-07-31: double (nullable = true)
 |-- 2001-08-31: double (nullable = true)
 |-- 2001-09-30: double (nullable = true)
 |-- 2001-10-31: double (nullable = true)
 |-- 2001-11-30: double (nullable =

In [ ]:
import pandas as pd

pandas_hpi_df = pd.read_excel("hpi_at_zip5.xlsx")
hpi_df = spark.createDataFrame(pandas_hpi_df)

hpi_df.show(5)

+-------------------+----+-----------------+------+------------------+------------------+
|Five-Digit ZIP Code|Year|Annual Change (%)|   HPI|HPI with 1990 base|HPI with 2000 base|
+-------------------+----+-----------------+------+------------------+------------------+
|               1001|1984|              NaN| 100.0|             52.31|             51.42|
|               1001|1985|            15.82|115.82|             60.59|             59.55|
|               1001|1986|            16.19|134.58|              70.4|              69.2|
|               1001|1987|            21.66|163.73|             85.65|             84.19|
|               1001|1988|            16.97|191.52|            100.19|             98.48|
+-------------------+----+-----------------+------+------------------+------------------+
only showing top 5 rows



In [ ]:
renamed_hpi_df = hpi_df.withColumnRenamed("Five-Digit ZIP Code", "ZIP")
renamed_hpi_df.printSchema()

root
 |-- ZIP: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Annual Change (%): double (nullable = true)
 |-- HPI: double (nullable = true)
 |-- HPI with 1990 base: double (nullable = true)
 |-- HPI with 2000 base: double (nullable = true)



In [ ]:
filtered_hpi_df = renamed_hpi_df.filter(hpi_df.Year.between(2000, 2023))

In [ ]:
filtered_hpi_df.show(5)

+----+----+-----------------+------+------------------+------------------+
| ZIP|Year|Annual Change (%)|   HPI|HPI with 1990 base|HPI with 2000 base|
+----+----+-----------------+------+------------------+------------------+
|1001|2000|             4.97|194.48|            101.74|             100.0|
|1001|2001|             6.25|206.64|            108.09|            106.25|
|1001|2002|             7.93|223.01|            116.66|            114.67|
|1001|2003|             8.49|241.94|            126.56|             124.4|
|1001|2004|            12.14|271.31|            141.93|            139.51|
+----+----+-----------------+------+------------------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql import functions as F

# Identify all date columns
date_cols = [c for c in housing_df.columns if c != "ZIP"]

# Build the stack() expression
# stack(n, col_name1, col_value1, ...) returns two columns (date, value)
expr = "stack({}, {}) as (Date, Value)".format(
    len(date_cols),
    ", ".join([f"'{c}', `{c}`" for c in date_cols])
)

# Apply the transformation
housing_df_unpivoted = housing_df.select("ZIP", F.expr(expr))

# Convert Date strings into actual dates
housing_df_unpivoted = housing_df_unpivoted.withColumn("Date", F.to_date("Date"))

housing_df_unpivoted.show(5)



+-----+----------+------------------+
|  ZIP|      Date|             Value|
+-----+----------+------------------+
|85142|2000-01-31|167370.94648678016|
|85142|2000-02-29|167391.21804895392|
|85142|2000-03-31| 167436.6693828767|
|85142|2000-04-30|167667.96006376506|
|85142|2000-05-31|168614.07141378833|
+-----+----------+------------------+
only showing top 5 rows



In [ ]:
housing_df_unpivoted.orderBy("ZIP").show(10)

+-----+----------+------------------+
|  ZIP|      Date|             Value|
+-----+----------+------------------+
|85003|2000-01-31|139551.56941663878|
|85003|2000-11-30|151215.76007809912|
|85003|2000-02-29|140155.28641971137|
|85003|2000-03-31|140998.74067297127|
|85003|2000-04-30|142921.41374430983|
|85003|2000-05-31| 144931.3380074081|
|85003|2000-06-30| 146280.1475486276|
|85003|2000-07-31|146964.32331888474|
|85003|2000-08-31| 147507.1966248258|
|85003|2000-09-30|148591.56394008014|
+-----+----------+------------------+
only showing top 10 rows



In [ ]:
# Ensure Year column exits
housing_df_with_year = housing_df_unpivoted.withColumn("Year", F.year("Date"))

# Compute the median (50th percentile) for each year
housing_df_median_yearly = (
    housing_df_with_year
    .groupBy("ZIP", "Year")
    .agg(F.expr("percentile_approx(Value, 0.5)").alias("Median_Value"))
    .orderBy("ZIP", "Year")
)

housing_df_median_yearly.show()

+-----+----+------------------+
|  ZIP|Year|      Median_Value|
+-----+----+------------------+
|85003|2000| 146280.1475486276|
|85003|2001|163394.25037429325|
|85003|2002|174426.95779830744|
|85003|2003| 189942.6267839241|
|85003|2004| 219160.8823153776|
|85003|2005| 291320.0792212078|
|85003|2006|363872.90554671764|
|85003|2007|362710.70239920955|
|85003|2008|315794.76809217274|
|85003|2009|235390.26030226567|
|85003|2010|217289.95822845167|
|85003|2011|181289.56809842392|
|85003|2012|196831.49921499102|
|85003|2013| 254630.4091002826|
|85003|2014| 294311.7592919077|
|85003|2015|303373.10751134396|
|85003|2016|333305.13877842267|
|85003|2017| 347330.2601775217|
|85003|2018|362289.90568724484|
|85003|2019|376347.98820858775|
+-----+----+------------------+
only showing top 20 rows



In [ ]:
filtered_hpi_matched = (
    filtered_hpi_df.join(housing_df_median_yearly, on=["ZIP", "Year"], how="inner")
)

In [ ]:
filtered_hpi_matched.printSchema()
filtered_hpi_matched.show()

root
 |-- ZIP: long (nullable = true)
 |-- Year: long (nullable = true)
 |-- Annual Change (%): double (nullable = true)
 |-- HPI: double (nullable = true)
 |-- HPI with 1990 base: double (nullable = true)
 |-- HPI with 2000 base: double (nullable = true)
 |-- Median_Value: double (nullable = true)

+-----+----+-----------------+------+------------------+------------------+------------------+
|  ZIP|Year|Annual Change (%)|   HPI|HPI with 1990 base|HPI with 2000 base|      Median_Value|
+-----+----+-----------------+------+------------------+------------------+------------------+
|85003|2000|             12.6|198.34|            211.88|             100.0| 146280.1475486276|
|85003|2001|            11.57|221.28|            236.39|            111.57|163394.25037429325|
|85003|2002|             4.58|231.43|            247.22|            116.68|174426.95779830744|
|85003|2003|             7.22|248.15|            265.08|            125.11| 189942.6267839241|
|85003|2004|            13.53|281.

In [ ]:
output_path = "housingDataComplete"
filtered_hpi_matched.coalesce(1).write.csv(
    path=output_path,
    header=True,
    mode="overwrite"
)